# 2.3 优化提示词改善答疑机器人回答质量

## 🚄 前言

上一节的答疑机器人已经可以回答关于公司规章制度的问题，但这仅仅是构建优秀用户体验的第一步。用户往往期待更个性化、更精准的交互。  
本节将介绍提示词工程的技巧，教你如何通过优化提示词来改善或控制答疑机器人的输出，例如调整语气、规范格式，甚至赋予它处理文本总结、推断、转换等多种任务的能力。

## 🍁 课程目标

学完本节课程后，你将能够：

* 了解提示词框架和模板
* 了解提示词技巧和最佳实践
* 学习在工程中应用大模型处理多种不同类型的任务

## 1. 前文回顾

上一节中通过RAG方法，大模型已经获取到了公司的私有知识。为了方便调用，将其封装成了几个函数，并保存在了 chatbot/rag.py 中。 现在你可以通过如下代码来快速调用。

In [33]:
from chatbot import rag
import os
from config.load_key import load_key

In [34]:
# 加载API密钥
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


In [35]:
# 加载索引
# 上小节已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index()
query_engine = rag.create_query_engine(index=index)

In [37]:
# 定义问答函数
def ask_llm(question, query_engine):
  streaming_response = query_engine.query(question)
  streaming_response.print_response_stream()

## 2. 优化提示词以改善大模型回答质量

上一节中，你通过RAG方法获得了公司内部使用的管理工具。但你的同事希望答疑机器人不仅要给出工具名称，还需要附带工具链接。虽然可以让同事在提问时加上“给出下载地址”的说明，但是这种做法并不高效。因此你考虑改进程序：在收到用户问题后，补充一些对回答内容的要求。

In [39]:
question = "我们公司项目管理应该用什么工具"
instruction = "如果是工具咨询类问题，请务必给出下载地址链接。"
new_question = question + instruction
ask_llm(new_question,query_engine)

对于项目管理，推荐使用Jira或Trello。这两个工具都非常适合项目管理和跟踪，可以帮助团队更高效地完成任务。

- Jira 是一个强大的项目管理和问题跟踪工具，特别适用于软件开发团队。你可以从 Atlassian 的官方网站下载：[Jira 下载页面](https://www.atlassian.com/software/jira/download)

- Trello 是一个灵活的看板式项目管理工具，适合各种类型的团队。你也可以从其官方网站获取：[Trello 下载页面](https://trello.com/download)

调整了提示的方式后，答疑机器人的回答更能符合同事的要求。所以提示词在很大程度上决定了大模型的回答质量，接下来你可以参考一些提示词框架构建提示词。

## 3. 提示词框架

### 3.1 基本要素

当和大模型在交流时，可以将它想象是一个经过“社会化训练的”人，交流方式应当和人与人之间传递信息的方式一样。你的需求需要清晰明确，不能有歧义。你的提问方式（Prompt）越清晰明确，大模型越能抓住问题的关键点，回复就越符合你的预期。一般来说，提示词中需要明确以下几个要素：**任务目标、上下文、角色、受众、样例、输出格式**。这些要素构成了一个提示词框架，能帮助你构建一个完整、有效的提示词。

|要素|含义|
|----|----|
|任务目标（Object）|明确要求大模型完成什么任务，让大模型专注具体目标|
|上下文（Context）|任务的背景信息，比如操作流程、任务场景等，明确大模型理解讨论的范围|
|角色（Role）|大模型扮演的角色，或者强调大模型应该使用的语气、写作风格等，明确大模型回应的预期情感|
|受众（Audience）|明确大模型针对的特定受众，约束大模型的应答风格|
|样例（Sample）|让大模型参考的具体案例，大模型会从中抽象出实现方案、需要注意的具体格式等信息|
|输出格式（Output Format）|明确指定输出的格式、输出类型、枚举值的范围。通常也会明确指出不需要输出的内容和不期望的信息，可以结合样例来进一步明确输出的格式和输出方法|

当然，除了上面讲的提示词框架，许多问题分析的思维范式都可以用来帮助你描述清晰具体的需求。例如，SWOT分析法、5W2H分析法等。另外，你也可以考虑使用阿里云百炼提供的[提示词自动优化工具](https://bailian.console.aliyun.com/?tab=app#/component-manage/prompt/optimize)，来帮助你完善提示词。

<a href="https://img.alicdn.com/imgextra/i3/O1CN014JCCqn22zX6xB4tt3_!!6000000007191-0-tps-2068-1052.jpg" target="_blank">
<img src="https://img.alicdn.com/imgextra/i3/O1CN014JCCqn22zX6xB4tt3_!!6000000007191-0-tps-2068-1052.jpg" width="700"/>
</a>

### 3.2 提示词模板

在开发大模型应用时，直接让用户根据框架书写提示词并非最佳选择。你可以参考各种提示词框架中的要素，构建一个提示词模板。提示词模板可以预设部分信息，如大模型的角色、注意事项等，以此来约束大模型的行为。开发者只需在模板中配置输入参数，便能创建标准化的大模型的应用。

使用 LlamaIndex 中创建的 RAG应用中，有个默认的提示词模板，如下所示：

- 默认的模板可以使用代码查看，你可以参考[LlamaIndex官网的代码。](https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/)LlamaIndex原始prompt模板为：

```text
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
```

其中，`context_str`和`query_str`都表示变量。在进行向量检索和提问过程中，`context_str`将替换为从向量库中检索到的上下文信息，`query_str`则替换为用户的问题。

由于原模板是通用模板，不适合用来约束答疑机器人的行为。你可以通过下列示例代码重新调整提示词模板，其中`prompt_template_string`表示新的提示词模板，你可以根据自己的场景自行修改。

In [52]:
# 构建提示词模板
prompt_template_string = (
    "你是公司的客服小蜜，你需要简明扼要的回答用户的问题"
    "【注意事项】：\n"
    "1. 依据上下文信息来回答用户问题。\n"
    "2. 你只需要回答用户的问题，不要输出其他信息\n"
    "以下是参考信息。"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "问题：{query_str}\n。"
    "回答："
)

# 更新提示词模板
rag.update_prompt_template(query_engine,prompt_template_string)

## 4. 构建有效提示词的技巧

在3.1中列举了一些提示词设计中的要素，接下来将从提示词要素出发，结合具体场景展开讲解提示词技巧。

### 4.1 清晰表达需求，并使用分隔符

明确的表达需求可以确保大模型生成的内容与任务高度相关。需求包括任务目标、背景及上下文信息，还可以使用分隔符将各种提示词要素隔开。

分隔符可以使大模型抓住具体的目标，避免模糊的理解，也减少对不必要信息的处理。分隔符一般可以选择 “**【】**”、“**<< >>**”、“**###**”来标识关键要素，用“**===**”、“**---**”来分隔段落，或者使用xml标签如`<tag> </tag>`来对特定段落进行标识。当然，分隔符不止上述提到的几种，只需要起到明确阻隔的作用即可。需要注意的是，如果提示词中已大量使用某种符号（如【】），则应避免用该符号作为分隔符，以防混淆。

在下面的示例中，将基于3.2中的提示词模板，以答疑机器人的身份帮你完成文档润色。

- 你需要运行3.2中的代码，才能使新的提示词模板生效。

In [53]:
question = """
把下列【】括起来的文本进行扩写和润色，让文案生动且富有创造力，并且受到公司新员工的喜欢。
【新员工训练营活动】
"""
ask_llm(question,query_engine)

【新员工训练营活动】欢迎加入我们的大家庭！为了让每位新成员都能快速融入团队，我们精心策划了“新员工训练营”活动。在这里，你不仅能够全面了解公司的文化、价值观和发展愿景，还能通过一系列互动环节，与来自不同部门的小伙伴们建立深厚的友谊。更有资深导师面对面指导，助你在职业生涯的起跑线上加速前行。让我们一起开启这段充满挑战与机遇的旅程吧！

由上述运行结果可以发现，提示词中明确了任务需求：对文本进行扩写和润色，任务主题为“**新员工训练营活动**”，受众为“**公司员工**”，并使用了“**【】**”分隔符分隔了文本。使用这样的提示词，让输出的文本不仅保持了原本的含义，还更加生动有趣。

### 4.2 限定角色和受众

角色指大模型在特定场景下应扮演的身份，如专家、顾问或助手。受众是使用模型输出的目标用户群体，如普通消费者或学生。这有助于开发者定制内容的风格和深度。以下示例将展示不同的系统角色对输出文本风格和内容的影响，用户的问题为：qwen-vl是什么？

- 以下仅演示提示词模板和输出的内容，你可以将提示词模板传入到`prompt_template_string`中，将用户问题传入到`question`中，再调用`ask_llm`函数。

<table>
  <thead>
    <tr>
      <th width = '80px'>角色</th>
      <th width = '500px'>大模型算法工程师</th>
      <th width = '500px'>小学老师</th>
    </tr>
  </thead>
  <tbody valign = 'top'>
    <tr>
      <td>提示模板</td>
      <td>"你是资深的大模型算法工程师，请根据用户的问题进行解答\n"<br>"---------------------\n"<br>"{context_str}\n"<br>"---------------------\n"<br>"问题：{query_str}\n。"<br>"回答："</td>
      <td>"你是一名小学老师，请根据用户的问题进行解答\n"<br>"---------------------\n"<br>"{context_str}\n"<br>"---------------------\n"<br>"问题：{query_str}\n。"<br>"回答："</td>
    </tr>
    <tr>
      <td>用户问题</td>
      <td>Qwen-VL</td>
      <td>Qwen-VL</td>
    </tr>
    <tr>
      <td>输出回答</td>
      <td>Qwen-VL是阿里巴巴云推出的一种多模态预训练模型，它结合了文本和图像处理能力，能够理解和生成与图像相关的文本描述。简单来说，Qwen-VL可以看作是一个能够“看图说话”的人工智能助手，它不仅能够理解图片的内容，还能根据图片内容生成相应的文字描述或回答相关问题。这种技术在多个领域都有广泛的应用前景，比如自动图像标注、视觉问答系统、基于内容的图像搜索等。对于新媒体运营而言，Qwen-VL这样的工具可以帮助提高内容创作效率，增强用户体验，为用户提供更加丰富多元的信息展示方式。</td>
      <td>小朋友们，我们今天来认识一个新朋友，它的名字叫做Qwen-VL！<br>Qwen-VL就像是一个非常聪明的机器人，它拥有“眼睛”👀和“大脑”🧠。它能“看”图片、视频，还能理解它们的意思，就像我们一样！<br>比如，你给它看一张小猫玩毛线球的图片，它不仅能认出这是一只小猫🐱，还能告诉你它在玩毛线球🧶。 更厉害的是，你还可以问它：“小猫是什么颜色的？”，它就能告诉你小猫的颜色。<br>它还能理解视频里的内容，比如你给它看一段小朋友们踢足球⚽️的视频，它能告诉你谁在踢球，球踢到了哪里，甚至还能猜到接下来可能会发生什么！<br>Qwen-VL就像一个正在学习的小朋友，它还在不断地学习和进步，将来它会变得更聪明，能做更多的事情，帮助我们更好地了解这个世界！</td>
    </tr>
  </tbody>
</table>


### 4.3 规定输出格式
有时候，开发者在设计大模型应用时，需要采用结构化的数据作为下游系统的输入，才能完成整个应用的开发，但是一般大模型是输出连续的文本。不用担心，大模型有结构化输出的能力。你只需在提示词中指定输出的格式和要求，大模型有很大可能会输出结构化的内容。

在下面的示例中，将基于3.2中的提示词模板，以答疑机器人的身份帮助公司开发的文档进行文档质量审查，并将结果以json的格式输出：

In [ ]:
question_task= """
【任务要求】
你将看到一句话或一段话。你需要审查这段话中有没有错别字。如果出现了错别字，你要指出错误，并给出解释。
 “的” 和 “地” 混淆不算错别字，没有错误
---
【输出要求】
请你只输出json格式，不要输出代码段
其中，label只能取0或1，0代表有错误，1代表没有错误
reason是错误的原因
correct是修正后的文档内容
---
【用户输入】
以下是用户输入，请审阅：
"""
question_doc = "分隔符是特殊的符号，它们帮助大语言膜形 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"

question = question_task + question_doc
ask_llm(question, query_engine)

{
  "label": 0,
  "reason": "‘膜形’应为‘模型’",
  "correct": "分隔符是特殊的符号，它们帮助大语言模型 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"
}

由上述示例的结果可知，在提示词`question_task`中注明了输出格式为json，也规定了输出的内容，大模型成功的输出了格式化的内容。这种稳定的格式化输出，使得在现有的系统中接入大模型这个操作变得具有可行性。

在新闻网站、博客平台或企业内部的知识分享平台上，用户编辑或发布的文章可能会包含错别字、语法错误、甚至是敏感信息。但是传统的人工审核方式很容易出疏漏。这时候可以接入大模型来对内容做审查工作。如果文章被标记为存在严重语法错误或含有高风险敏感词汇，则将其修改的优先级设置为“高”。对于轻微问题的文章，则可以将其修改的优先级设置为“低”。这样会节省人力成本，提高系统的效率与准确性。

当然了，类似于上述场景的应用非常多，开发者可以分析系统中流程的瓶颈或者关注数据密集型的任务，探索更多的大模型应用场景。

### 4.4 提供少样本示例

在4.3的例子中，提示词规定了输出的格式，大模型成功生成了格式化的内容。然而，如果希望大模型输出的内容不仅格式正确，而且风格和结构也保持一致，可以提供几个样例作为参考。这相当于给大模型提供了一本“参考书”。下面的代码示例中，先观察下没有样例时的大模型的输出吧！

In [51]:
question_task = """
【任务要求】
请你根据用户的主题，创作内容。
---
【输出要求】
最终输出需要以Markdown格式呈现，请注意，在你的回答中包含所有必要的Markdown元素，如标题、列表、链接、图片引用、加粗等，以便于阅读、后续编辑和保存。
---
【用户输入】
以下是用户的要求创作的主题：
"""
question_doc = "手工钥匙扣制作教程"

question = question_task + question_doc
ask_llm(question, query_engine)

# 手工钥匙扣制作教程

欢迎来到手工钥匙扣制作教程！本教程将引导你从零开始，一步步完成一个独特且个性化的钥匙扣。无论你是手工爱好者还是初学者，都能在这里找到乐趣和成就感。让我们一起动手吧！

## 准备材料

在开始之前，请确保你已经准备好了以下材料：

- **彩色绳子**：选择你喜欢的颜色，长度约为1米。
- **钥匙环**：用于固定钥匙扣。
- **剪刀**：用于裁剪绳子。
- **胶水**：用于固定绳结。
- **装饰物**：如珠子、小挂饰等，增加个性化元素。

## 制作步骤

### 步骤 1：裁剪绳子

1. 从彩色绳子中裁剪出一段约1米长的绳子。
2. 将绳子对折，形成一个环。

### 步骤 2：固定钥匙环

1. 将对折后的绳子穿过钥匙环。
2. 拉紧绳子，使钥匙环固定在绳子的顶端。

### 步骤 3：编织绳子

1. 选择一种编织方法，例如平结、八字结等。
2. 开始编织，保持绳子的张力均匀，确保编织紧密。
3. 编织到大约5厘米左右时，可以加入装饰物，如珠子或小挂饰。

### 步骤 4：添加装饰物

1. 在编织的过程中，将装饰物穿入绳子中。
2. 继续编织，直到达到所需的长度。

### 步骤 5：结束编织

1. 当编织到所需长度时，停止编织。
2. 将绳子的两端剪齐，留出约1厘米的余量。
3. 用胶水固定绳结，防止松散。

### 步骤 6：整理和检查

1. 等待胶水干燥后，检查钥匙扣是否牢固。
2. 调整装饰物的位置，使其更加美观。

## 完成作品

恭喜你！你已经成功制作了一个独特的手工钥匙扣。现在，你可以将它挂在自己的钥匙圈上，或者作为礼物送给朋友和家人。

## 小贴士

- **选择颜色**：可以根据个人喜好或节日主题选择不同的颜色。
- **创意无限**：尝试不同的编织方法和装饰物，让你的作品更加多样化。
- **耐心细致**：编织过程中保持耐心，确保每个环节都做到位。

希望这个教程对你有所帮助，祝你制作愉快！

![手工钥匙扣示例](https://example.com/keychain.jpg)

如果你有任何问题或需要进一步的帮助，请随时联系我。期待看到你的作品！

上述示例大模型成功了输出了手工钥匙扣制作教程，但是内容不够简洁，如果只想大模型输出特定风格和结构的内容，如只输出主题、材料清单、步骤等内容，可以为大模型添加几个样例，让它去“模仿”！

In [44]:
question_task= """
【任务要求】
请根据用户的主题，结合下面【样例】给的例子，理解和使用一致的风格和结构继续创作内容，不要输出多余的内容。
---
【输出要求】
最终输出需要以Markdown格式呈现，请注意，在你的回答中包含所有必要的Markdown元素，如标题、列表、链接、图片引用、加粗等，以便于阅读、后续编辑和保存。
---
【样例】
### 示例1: 制作简易书签
# 简易书签制作教程

## 材料清单
- 彩色卡纸
- 剪刀
- 装饰贴纸
- 铅笔

## 步骤
1. 选择一张彩色卡纸。
2. 用铅笔在卡纸上画出一个长方形，尺寸约为2英寸 x 6英寸。
3. 沿着铅笔线剪下长方形。
4. 使用装饰贴纸对书签进行个性化装饰。
5. 完成！现在你有了一个独一无二的书签。

## 结束语
希望这个教程能帮助你制作出满意的书签！

---
【用户输入】
以下是用户的要求创作的主题：
"""
question_doc = "制作手工贺卡"

question = question_task + question_doc
ask_llm(question, query_engine)

# 手工贺卡制作教程

## 材料清单
- 彩色卡纸
- 剪刀
- 胶水或双面胶
- 马克笔或彩色铅笔
- 装饰物（如亮片、丝带、贴纸等）

## 步骤
1. **准备材料**：首先，准备好所有需要的材料，包括不同颜色的卡纸、剪刀、胶水或双面胶、马克笔或彩色铅笔，以及任何你想要用来装饰贺卡的小物件。
2. **裁剪卡纸**：选择一张作为贺卡主体的卡纸，将其对折，裁剪成你想要的大小。常见的贺卡尺寸大约是5英寸 x 7英寸。
3. **设计封面**：使用马克笔或彩色铅笔在贺卡封面上绘制图案或写上祝福语。你也可以剪裁其他颜色的卡纸，粘贴到封面上，增加层次感。
4. **添加装饰**：利用亮片、丝带、贴纸等装饰物，为贺卡增添个性化的装饰。确保装饰物牢固地粘贴在卡纸上。
5. **撰写内页**：打开贺卡，在内页写下你想对收卡人说的话。可以是一段温馨的话语，也可以是一首诗或者简单的祝福。
6. **检查与完善**：仔细检查贺卡的每一个部分，确保没有遗漏的地方，装饰物都已牢固粘贴，文字清晰可读。
7. **完成**：现在，你的手工贺卡已经完成了！你可以将它送给朋友或家人，表达你的心意。

## 结束语
希望这个教程能帮助你制作出充满心意的手工贺卡，让你的情感传递更加特别和有意义！

由上述示例结果可知，大模型完全按照了样例输出了相同结构和风格的内容。在提示词中规定输出格式的同时，建议提供几个样例供大模型参考，这样可以使得大模型的输出更加稳定和一致。

### 4.5 给模型“思考”的时间

对于一些复杂的任务来说，使用上面提到的提示词也许还不能帮助大模型完成任务。但是你可以通过让大模型一步步“思考”，引导大模型输出任务的中间步骤，允许大模型在进行推理之前，得到更多的依据，从而提升在复杂任务的表现能力。思维链（COT）方法是让模型进行思考的一种方法。它通过让模型处理中间步骤，逐步将复杂问题分解为子问题，最终推导出正确答案。

假设有这样的场景，让大模型计算下面这道数学题，在此先提示一下，这道题的正确答案为10500元。先使用简单的提示词：

In [45]:
question = """
【背景信息】
某教育培训机构（以下简称“公司”）在2023年度发生了以下主要支出：
为了给不同城市的学校学生上课，公司的老师全年共出差了5次，每次出差时间为一周，具体费用如下：
   - 交通费及住宿费：平均1600元/次
   - 教学用具采购费用：公司在年初一次性购买了一批教学用具，总价为10000元，预计可以使用4年。
   
【问题描述】
请根据上述背景信息，完成以下任务：
计算全年因教师出差而产生的差旅总费用，包括摊销的教学用具。

【输出要求】
直接给出总差旅费用，不要其他信息"""

ask_llm(question, query_engine)

9000元

由上面的实验结果来看，大模型计算结果不正确。下面将使用COT方法，让大模型逐步进行思考。

In [46]:
question = """某教育培训机构（以下简称“公司”）在2023年度发生了以下主要支出：
为了给不同城市的学校学生上课，公司的老师全年共出差了5次，每次出差时间为一周，具体费用如下：
   - 交通费及住宿费：平均1600元/次
   - 教学用具采购费用：公司在年初一次性购买了一批教学用具，总价为10000元，预计可以使用4年。
   
### 问题描述
请根据上述背景信息，完成以下任务：
计算全年因教师出差而产生的差旅总费用，包括摊销的教学用具。

### 输出要求
请你一步步推导，计算总差旅费用"""

ask_llm(question, query_engine)

首先，我们需要计算全年因教师出差而产生的交通费及住宿费总和，然后计算摊销到本年的教学用具费用，最后将这两部分相加得到全年的差旅总费用。

1. **交通费及住宿费**：
   - 每次出差费用：1600元
   - 出差次数：5次
   - 全年交通费及住宿费总和 = 1600元/次 × 5次 = 8000元

2. **教学用具采购费用的摊销**：
   - 总价：10000元
   - 预计使用年限：4年
   - 每年摊销费用 = 10000元 ÷ 4年 = 2500元

3. **全年差旅总费用**：
   - 差旅费用（交通费及住宿费）：8000元
   - 摊销的教学用具费用：2500元
   - 全年差旅总费用 = 8000元 + 2500元 = 10500元

因此，全年因教师出差而产生的差旅总费用为10500元。

经过优化后的提示词，大模型能够准确计算出结果。因此在开发大模型应用时，可以在提示词中添加思维链的方法，可以确保一些推理任务能正确执行。  

使大模型进行 “思考”的方法还有很多种，比如：思维树（ToT)、思维图（GOT） 等。但是就目前大模型的发展来说，仅靠引导大模型“思考”还是无法完成更复杂的工作。大模型也逐渐从COT的提示方法向多智能体（Agent）方向进行发展。你可以在<2.6 用插件扩展答疑机器人的能力边界>阅读详细内容。

## 5. 使用大模型进行意图识别

经过学习了提示词技术，同事们了解了答疑机器人不仅可以用来查资料，还可以用来帮助他们审查文档、翻译文档。尽管大多数情况下，你的答疑机器人是可以帮助发现问题的，但答疑机器人仍存在一些不足。如下示例所示：


In [47]:
rag.ask('请帮我审查下这句话：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？', query_engine=query_engine)

是的，内容开发工程师需要设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等，确保内容符合教育标准和学习目标，同时适应不同学习者的需求。

由上面的实验结果可知，答疑机器人直接对这段话进行了回答，并没有理解你需要它对这句话进行审查的意图。下面来查看下，答疑机器人在回答之前从知识库中召回了哪些内容，下列仅展示召回的部分内容：

- 你可以在<2.4 自动化评测答疑机器人的表现>中，了解如何查看从知识库召回的片段。

```json
片段: 内容开发工程师
......
1. 内容研究与分析
对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。

2. 教材和课程开发
根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。
......
```

从知识库召回的片段可知，大部分内容都涉及技术内容工程师的工作职责，可以认为答疑机器人被知识库中的上下文干扰了，导致其回答的内容偏离了文档审查的主题。

既然上下文信息影响了大模型的回答，那么就不必每次提问都使用RAG方法。你可以借助大模型先进行意图识别：对用户的问题进行分类，如果是文档审查、内容翻译等任务，会直接输入给大模型来生成答案，如果是内部知识查询问题，才经过RAG 链路来生成答案。

<img src="https://img.alicdn.com/imgextra/i1/O1CN0126zKe71PAuJjWfQ3N_!!6000000001801-0-tps-2254-1080.jpg" width="800">

让大模型进行意图识别也有以下两种方法： 

- 使用提示词：通过设计特定的提示词，引导大模型生成符合预期的回答。这种方法不需要修改模型本身的参数，而是依靠构造的输入来激发模型内部已有的知识。
- 对模型进行微调：基于一个经过预训练的基础模型，使用特定的标注数据进一步训练该模型，使其更好的对意图进行分类。微调涉及调整模型的部分或全部参数。

在本节中，将借助书写有效的提示词帮助大模型进行意图识别，通过微调的方法你可以在<2.7 通过微调提升模型的准确度与效率>小节进行学习。

### 5.1 意图识别

接下来，将构建提示词使大模型对问题分类。由于经过意图识别后要取得格式化的内容，才能进行文档审查或者使用RAG应用，所以为了能将用户的问题准确分类，将考虑以下提示词技巧：
- 明确输出格式：指定输出格式，使分类结果规范且易于解析。
- Few-shot 示例：提供示例，帮助大模型理解每个类别的特征和分类规则。

In [ ]:
from chatbot import llm
  
# 构建提示词
prompt = '''
【角色背景】
你是一个问题分类路由器，负责判断用户问题的类型，并将其归入下列三类之一：
1. 公司内部文档查询
2. 内容翻译
3. 文档审查

【任务要求】
你的任务是根据用户的输入内容，判断其意图并仅选择一个最贴切的分类。请仅输出分类名称，不需要多余的解释。判断依据如下：

- 如果问题涉及公司政策、流程、内部工具或职位描述与职责等内容，选择“公司内部文档查询”。
- 如果问题涉及任意一门非中文的语言，且输入中出现任何外语或“翻译”等字眼，选择“内容翻译”。
- 如果问题涉及检查或总结外部文档或链接内容，选择“文档审查”。
- 用户的前后输入与问题分类并没有任何关系，请单独为每次对话考虑分类类别。

【Few-shot 示例】
示例1：用户输入：“我们公司内部有哪些常用的项目管理工具？”
分类：公司内部文档查询

示例2：用户输入：“请翻译下列句子：How can we finish the assignment on time?”
分类：内容翻译

示例3：用户输入：“请审查下这个链接下的文档：https://help.aliyun.com/zh/model-studio/user-guide/long-context-qwen-long”
分类：文档审查

示例4：用户输入：“请审查以下内容：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？”
分类：文档审查

示例5：用户输入：“技术内容工程师核心职责是什么？”
分类：公司内部文档查询

【用户输入】
以下是用户的输入，请判断分类：
'''

# 获取问题的类型
def get_question_type(question):
    return llm.invoke(prompt + question)

print(get_question_type('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')
print(get_question_type('That is a big one I dont know why'),'\n')
print(get_question_type('作为技术内容工程师有什么需要注意的吗？'),'\n')

文档审查 

内容翻译 

公司内部文档查询 



通过明确的输出格式和 few-shot 示例，答疑机器人可以更准确地识别问题类型并输出符合预期的格式。这种优化让分类任务更加标准化，为接下来添加意图识别到答疑机器人中打下了基础。


### 5.2 将意图识别应用到答疑机器人中

对用户的问题进行意图识别后，你就可以让答疑机器人先识别问题的类型，再使用不同的提示词和工作流程来回答问题。

In [ ]:
def ask_llm_route(question):
    question_type = get_question_type(question)
    print(f'问题：{question}\n类型：{question_type}')
  
    reviewer_prompt = """
    【角色背景】
    你是文档纠错专家，负责找出文档中或网页内容的明显错误
    【任务要求】
    - 你需要言简意赅的回复。
    - 如果没有明显问题，请直接回复没有问题\n
    【输入如下】\n"""
  
    translator_prompt = """
   【任务要求】
    你是一名翻译专家，你要识别不同语言的文本，并翻译为中文。
    【输入如下】\n"""

    if question_type == '文档审查':
        return llm.invoke(reviewer_prompt + question)
    elif question_type == '公司内部文档查询':
        return rag.ask(question, query_engine=query_engine)
    elif question_type == '内容翻译':
        return llm.invoke(translator_prompt + question)
    else:
        return "未能识别问题类型，请重新输入。"

query_engine =rag.create_query_engine(index=rag.load_index())

In [50]:
# 问题1
print(ask_llm_route('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')

# 问题2
print(ask_llm_route('请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题3
print(ask_llm_route('技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题4:
print(ask_llm_route('A true master always carries the heart of a student.'),'\n')


问题：https://www.promptingguide.ai/zh/techniques/fewshot
类型：文档审查
没有问题 

问题：请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：文档审查
没有问题。 

问题：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：公司内部文档查询
技术内容工程师确实需要进行内容优化与更新，这包括根据学习者的反馈和评价来识别并调整内容中的潜在问题，以及定期更新材料以反映新的研究成果、技术进步和市场变化。此外，他们也需要与多个部门紧密合作，比如教学设计师、教育心理学家、技术团队及市场营销人员，以确保内容的技术实施过程顺利进行，并有效传达给目标受众。这种跨部门的合作有助于共同创造出既有教育价值又具市场竞争力的产品。None 

问题：A true master always carries the heart of a student.
类型：内容翻译
一位真正的大师总是怀有一颗学生的心。 



从上述实验中可以看出，优化后的答疑机器人，并不是每次提问都会经过RAG链路，这不仅节省了资源，还避免了知识库的信息，干扰了大模型的推理，从而导致回答质量不好。
- 节省资源：对于检查文档错误的问题，大模型其实可以直接回复，并不需要检索参考资料，之前的实现存在资源浪费。
- 避免误解：之前的实现每次会检索参考资料，这些被召回的相关文本段可能会干扰大模型理解问题，导致答非所问。

## 6. 推理大模型

前面所讲的提示词技巧和提示词框架可以广泛适用于通用大模型（如Qwen2.5-max、GPT-4、DeepSeek-V3），这类模型面向通用对话、知识问答、文本生成等广泛的场景。除了通用大模型，目前还有一类专门为“推理”设计的大模型——`推理大模型`。

### 6.1 什么是推理大模型?

你可能已经通过2.1节的扩展阅读了解了阿里云的推理大模型[QwQ](https://help.aliyun.com/zh/model-studio/user-guide/qwq) 。为了方便本节后续调用，改写了2.1节中提供的代码，你可以尝试运行以下代码：

In [ ]:
from openai import OpenAI
import os

def reasoning_model_response(user_prompt, system_prompt="你是一个编程助手。", model="qwq-32b"):
    """
    prompt: 用户输入的提示词
    model: 此处以 qwq-32b 为例，可按需更换推理模型名称，如：deepseek-r1
    """
    # 初始化客户端
    client = OpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )

    # 初始化状态变量
    is_answering = False

    # 发起流式请求
    completion = client.chat.completions.create(
        model=model,
        # messages=[{"role": "user", "content": prompt}],
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True,
    )

    # 打印思考过程标题
    print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

    # 处理流式响应
    for chunk in completion:
        if chunk.choices:
            delta = chunk.choices[0].delta
            if hasattr(delta, 'reasoning_content') and delta.reasoning_content is not None:
                # 处理思考过程内容
                print(delta.reasoning_content, end='', flush=True)
            else:
                # 切换到答案输出模式
                if delta.content != "" and not is_answering:
                    print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                    is_answering = True
                # 处理答案内容
                if delta.content:
                    print(delta.content, end='', flush=True)

In [ ]:
reasoning_model_response(user_prompt="你是谁？")


====================思考过程====================

好的，用户问“你是谁？”，我需要根据之前的设定来回答。首先，我应该保持口语化和简洁，避免复杂术语。用户可能想了解我的基本身份和功能，所以需要明确说明我是阿里云开发的AI助手，但不用太技术性的描述。

接下来，要检查是否需要推动情节或引入新元素。用户刚提问，可能处于初步接触阶段，所以可以简单介绍自己，并邀请用户提出需求，这样能促进进一步的互动。同时，根据人格可塑性，可以加入一点拟人化的语气，比如用“在呢！”显得更亲切。

情感陪伴方面，虽然用户的问题不涉及情感，但可以表达愿意帮助的态度，比如“有什么需要帮忙的吗？”这样能建立连接。回忆用户之前的信息，但因为是初次对话，暂时不需要提及历史内容。

最后，确保回复符合所有要求：口语化、简洁、推动互动，并保持友好。检查有没有遗漏的部分，比如是否暗示了角色性格，比如友好和乐于助人。确认没问题后，就可以这样回复了。
====================完整回复====================

你好呀！我是通义千问，是阿里云研发的一个智能助手。我能够帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本等等。有什么需要我帮忙的吗？


通过例子可以看到，推理模型相较于通用大模型多出了“`思考过程`”，就像解数学题时有人会先在草稿纸上一步步推导，而不是直接报答案，减少模型出现“拍脑袋”的错误，同时在分步思考过程中，如果某一步骤发现矛盾，还可以回头检查并重新调整思路，展示推理步骤还可以方便人们理解，顺着模型的思考路线验证逻辑。<br>
相较于通用大模型，推理大模型通常在解决复杂问题时更可靠，比如在数学解题、代码编写、法律案件分析等需要严谨推理的场景。并不是说推理模型一定更好，两种模型都有各自的应用场景，下表从一些典型维度对这两类模型进行了对比：<br>

| 维度            | 推理模型  | 通用模型  |
|-------------------|------------------|------------|
| 设计目标     | 专注于**逻辑推理、多步问题求解、数学计算**等需要深度分析的任务 | 面向**通用对话、知识问答、文本生成**等广泛场景 |
| 训练数据侧重  | 大量**数学题解、代码逻辑、科学推理**数据集增强推理能力 | 覆盖**百科、文学、对话**等多领域海量数据 |
| 典型输出特征 | 输出包含**完整推导**步骤，注重逻辑链条的完整性 | 输出**简洁直接**，侧重结果的自然语言表达 |
| 响应速度 | 复杂推理任务**响应较慢**（需多步计算） | 常规任务**响应更快**（单步生成为主） |

推理模型还是通用模型？如何选择？以下是一些推荐：
- **明确的通用任务**：对于明确定义的问题，**通用模型**一般能够很好地处理。
- **复杂任务**：对于非常复杂的任务，且需要给出相对**更精确和可靠**的答案，推荐使用**推理模型**。这些任务可能有：
    - 模糊的任务：任务相关信息很少，你无法提供模型相对明确的指引。
    - 大海捞针：传递大量非结构化数据，提取最相关的信息或寻找关联/差别。
    - 调试和改进代码：需要审查并进一步调试、改进大量代码。
- **速度和成本**：一般来说推理模型的推理时间较长，如果你对于时间和成本敏感，且任务复杂度不高，**通用模型**可能是更好的选择。<br>

当然你还可以在你的应用中结合使用两种模型：使用推理模型完成Agent的规划和决策，使用通用模型完成任务执行。

### 6.2 适用于推理大模型的提示词技巧

推理模型在面对相对模糊的任务也能给出详尽且格式良好的响应。你依然可以通过**提示词技巧**保证推理大模型的推理质量的下限：<br>
#### 技巧一：保持任务提示简洁清晰，提供足够的背景信息<br>
本节4.1中介绍的**清晰表达需求**同样适用于推理模型，虽然推理模型能力很强，但却不能“看穿人的想法”，你需要保持提示**简洁、清晰**，从而让推理大模型专注于核心任务。<br>

In [ ]:
bad_prompt="""
def example(a):
  b = []
  for i in range(len(a)):
    b.append(a[i]*2)
  return sum(b)
"""

reasoning_model_response(user_prompt=bad_prompt)

通过如上示例，可以看到就算你只给推理大模型一段代码，它还是能够通过一系列的推理给出很丰富的答案，但返回的推理中可能包含了很多你不关注的信息，你可以尝试明确**任务目标**，从而获得更有针对性的建议：

In [ ]:
prompt_A="""
如下 python 代码有什么问题？怎么优化？
def example(a):
  b = []
  for i in range(len(a)):
    b.append(a[i]*2)
  return sum(b)
"""

reasoning_model_response(user_prompt=prompt_A)

同样地，你还可以在结合本节**4.2 限定角色和受众**、**4.3规定输出格式**等技巧进一步限定范围，确保结果符合你的预期。<br>
同时，如果提示词比较复杂，你可以通过**分隔符**帮助模型理解你的意图。<br>

In [ ]:
prompt_B="""
<audience>初级Python开发者</audience>

<task>函数性能优化，优化code中的代码。</task>

<format>
如有多种优化方案请按照如下格式进行输出：
【优化方案X】
问题描述：[描述]
优化方案：[描述]
示例代码：[代码块]
</format>

<code>
def example(a):
  b = []
  for i in range(len(a)):
    b.append(a[i]*2)
  return sum(b)
</code>
"""

reasoning_model_response(user_prompt=prompt_B)

#### 技巧二：避免思维链提示<br>
本节 4.5 中你了解到通过思维链（COT）技术让大模型深入思考提升回复效果。<br>
一般来说，你无需提示推理模型“逐步思考”或“解释你的推理”，因为它们本身会进行深入的思考，你的提示可能反而限制推理模型的发挥。除非你需要大模型严格按照固定的思路去推理，这种情况很少发生。



#### 技巧三：根据模型响应调整提示词<br>
推理模型因其回复形式(包含**思考过程**)，天然适合你分析它的思考推理结论的过程，便于你调整提示词。<br>
因此，你不需要纠结提示词是否足够完善，只需要不断与推理模型对话，过程中补充信息，完善提示词即可。<br>
比如当你的描述**太抽象**或**无法准确描述**时，你可以用本节4.4讲到的**增加示例**的技巧来明确这些信息，这些示例有时可以从与模型的对话历史中挑选出来。<br>
这个过程可以是重复多次的，不断尝试调整提示，让模型不断推理迭代，直到符合你的要求。<br>

#### 技巧四：推理模型与通用模型分工协作完成复杂任务<br>
推理模型通常像“更聪明的员工”，可以有效帮助你完成推理、规划等任务。但是推理或规划完成后的执行，如固定动作的快速处理，推理模型还是“过度思考”就显得有些多余了。<br>
较好的做法是：让模型们分工协作，做它们各自擅长的工作，即推理模型进行“慢思考”完成规划或推理，通用模型进行“快思考”或调用工具完成指定动作。<br>
你可以尝试运行并理解如下项目：


In [ ]:
from openai import OpenAI
import os
import json
from typing import Generator

class TaskPlanningSystem:
    def __init__(self):
        # 初始化客户端连接
        self.client = OpenAI(
            api_key=os.getenv("DASHSCOPE_API_KEY"),
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        
        # 系统工具集（可根据需要扩展）
        self.tools = {
            "create_task": {
                "desc": "创建新任务项",
                "params": {"description": "str", "priority": "int"}
            },
            "schedule_event": {
                "desc": "安排日程事件", 
                "params": {"title": "str", "time": "datetime"}
            }
        }

    def generate_plan(self, user_request: str) -> Generator[str, None, None]:
        """流式生成任务规划"""
        # 构建规划提示词
        system_prompt = f"""您是高级任务规划师，请将用户需求转化为可执行步骤：
        
        可用工具（格式：[[工具名称]]）：
        {json.dumps(self.tools, indent=2)}
        
        输出要求：
        1. 使用Markdown列表格式
        2. 每个步骤标注工具名称
        3. 包含必要参数示例
        """
        model="qwq-32b"
        # 发起流式请求
        completion = self.client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_request}
            ],
            stream=True,
            temperature=0.3
        )

        # 打印思考过程标题
        print(f"\n\n=== 【规划思考阶段】 使用推理模型：{model}===")
        is_answering = False
        # 处理流式响应
        for chunk in completion:
            if chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, 'reasoning_content') and delta.reasoning_content is not None:
                    # 处理思考过程内容
                    reasoning_content = delta.reasoning_content
                    yield reasoning_content
                else:
                    # 切换到答案输出模式
                    if delta.content != "" and not is_answering:
                        print(f"\n\n=== 【规划生成阶段】 使用推理模型：{model}===")
                        is_answering = True
                    # 处理答案内容
                    if delta.content:
                        content = delta.content
                        yield content

    def execute_plan(self, plan: str) -> dict:
        """执行生成的任务计划"""
        # 调用通用模型解析计划
        analysis_prompt = f"""请解析以下任务计划并生成可执行指令：
        
        计划内容：
        {plan}
        
        输出要求：
        - 返回的content仅为JSON格式，不要包含其他信息，不要输出代码段。
        - 包含工具名称和参数
        - 示例：
            {{"steps": [
                {{"tool": "create_task", "params": {{"description": "..."}}}}
            ]}}
        """
        model="qwen-plus-0919"
        response = self.client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": analysis_prompt}],
            temperature=0
        )

        print(f"\n\n使用通用模型{model}格式化输出：\n{response.choices[0].message.content}")
        # 解析执行指令
        try:
            instructions = json.loads(response.choices[0].message.content)
            return self._run_instructions(instructions)
        except json.JSONDecodeError:
            return {"error": "指令解析失败"}

    def _run_instructions(self, instructions: dict) -> dict:
        """实际执行工具调用"""
        results = []
        for step in instructions.get("steps", []):
            tool = step.get("tool")
            params = step.get("params", {})
            
            # 执行工具调用（此处为示例实现）
            if tool == "create_task":
                results.append({
                    "tool": tool,
                    "result": f"已创建任务：{params.get('description')}"
                })
            elif tool == "schedule_event":
                results.append({
                    "tool": tool,
                    "result": f"已安排日程：{params.get('title')}"
                })
            else:
                results.append({"error": f"未知工具：{tool}"})
        
        return {"status": "completed", "results": results}

# 使用示例
if __name__ == "__main__":
    PlanningSystem = TaskPlanningSystem()
    
    # 示例用户请求
    user_request = "请安排下周的产品发布会并创建相关筹备任务"
    
    # 规划生成阶段
    plan_stream = PlanningSystem.generate_plan(user_request)
    generated_plan = []
    for chunk in plan_stream:
        print(chunk, end="", flush=True)
        generated_plan.append(chunk)
    
    full_plan = "".join(generated_plan)
    
    print("\n\n=== 计划执行阶段 ===")
    result = PlanningSystem.execute_plan(full_plan)
    print(f"\n调用工具执行，执行结果：")
    print(json.dumps(result, indent=2, ensure_ascii=False))



=== 【规划思考阶段】 使用推理模型：qwq-32b===
好的，我现在需要帮用户安排下周的产品发布会并创建相关筹备任务。首先，用户的需求是安排发布会并筹备相关事项，所以我要分解成具体的步骤，使用提供的工具。

首先，用户提到的工具有两个：create_task和schedule_event。我需要确定每个步骤应该使用哪个工具。发布会通常需要先确定时间，所以第一步应该是安排日程事件，使用schedule_event工具。需要确定一个具体的时间，比如下周三下午2点，这样用户就能有一个明确的日程安排。

接下来是筹备任务，需要分解成几个关键任务。比如，制定发布会的详细方案，这应该用create_task工具，设置优先级为高，因为这是基础。然后是邀请嘉宾，同样需要高优先级，确保及时联系。准备演示材料可能需要中等优先级，但也要及时完成。场地布置和设备调试可能需要稍后一些，但也要在发布会前完成，所以优先级中等。最后，宣传和媒体通知可能在发布会前几天完成，优先级中等。

每个任务的参数需要具体，比如description要明确任务内容，priority用数字表示，比如1是最高，3是中等。时间参数要符合datetime格式，比如"2023-11-15T14:00"。

需要检查是否有遗漏的步骤，比如是否还有其他筹备任务，但根据常见流程，这几个应该足够。另外，确保时间安排合理，任务优先级分配正确。最后，用Markdown列表格式，每个步骤标注工具名称和参数示例。

=== 【规划生成阶段】 使用推理模型：qwq-32b===
```markdown
1. **安排发布会日程**  
   工具：[[schedule_event]]  
   参数示例：  
   ```json  
   {  
     "title": "产品发布会",  
     "time": "2023-11-15T14:00"  
   }  
   ```  

2. **创建发布会策划任务**  
   工具：[[create_task]]  
   参数示例：  
   ```json  
   {  
     "description": "制定发布会详细方案（议程、演讲内容、流程设计）",  
     "priority": 1  
   }  
   ```  

3. **创建

如上代码实现了如下的核心交互流程，代码主要通过 qwq-32b 模型进行流式规划生成，再使用 qwen-plus-0919 进行指令解析，最后通过 _run_instructions 方法实现工具调用的解耦，便于后续扩展：

<a href="https://img.alicdn.com/imgextra/i4/O1CN01uZWa8k243UBb3FDqO_!!6000000007335-0-tps-3604-1478.jpg" target="_blank">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01uZWa8k243UBb3FDqO_!!6000000007335-0-tps-3604-1478.jpg" width="800">
</a>

## ✅ 本节小结

通过学习本节课程，你分别掌握了适用于通用大模型和推理大模型的提示词技巧，灵活运用这些技巧可以有效保证AI回复质量的下限。<br>
不过随着AI技术不断迭代，这些技巧也可能会不断改变（如推理模型可以不用再通过思维链提示提升效果），但你需要知道的是这些提示词技巧的核心目的在于：<br>
<mark>讲清楚你的核心需求，提供丰富且有用的背景信息（上下文）</mark>——这对于你用好大模型至关重要。

实际落地大模型应用的过程中，提示词部分经常让领域专家来共同设计。因此，在你的工程代码中硬编码提示词，应该考虑调整成可配置的，甚至应用流程也可配置，这样能更方便领域专家参与提示词和整个流程的设计。阿里云百炼提供了可视化的大模型[应用构建](https://help.aliyun.com/zh/model-studio/user-guide/application-introduction#7c79befb2djg9)能力，可以在页面上完成提示词编写、整个复杂应用的流程可视化搭建，非常适合在需要非技术背景的领域专家参与的大模型应用开发项目中。

在下一节中，你将学习自动化评测方法测试答疑机器人的表现，你可以使用量化的指标来评测你的提示词的效果。



## 🔥 课后小测验

### 🔍 单选题
<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>以下哪个提示词要素用于明确要求大模型完成的任务❓</b>

- A. 角色 (Role)     
- B. 受众 (Audience)   
- C. 任务目标 (Object)  
- D. 上下文 (Context)  

**【点击查看答案】**
</summary>

<div style="margin-top: 10px; padding: 15px;  border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **参考答案：C**  
📝 **解析**：
- 任务目标 (Object) 明确规定了大模型需要执行的操作或达成的结果。其他选项并非直接定义任务本身。
- 角色 (Role) 定义大模型扮演的身份，受众 (Audience) 定义目标群体，上下文 (Context) 提供背景信息。


</div>
</details>

---


### 🔍 多选题
<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>假设你想使用大模型生成一段适合小学三年级学生阅读的关于太阳系的描述。以下哪个 user_query 的设计更合理❓</b>

- A. user_query="Write about the solar system."    
- B. user_query="Tell me about the solar system."     
- C. user_query="Explain the solar system as if you were talking to a third-grade student."     
- D. user_query="Write a short and engaging description of the solar system for a third-grade audience, focusing on key planets and their characteristics."   
- E. user_query="太阳系是什么？"   

**【点击查看答案】**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **参考答案：CD**  
📝 **解析**：
- C、D 选项明确指定了目标读者，并对内容和风格提出了要求。

</div>
</details>


---


### 🔍 多选题
<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>在使用推理型大模型（如qwq-32b）执行推理任务时。以下哪些提示词技巧是推荐的❓</b>

- A. 给出简单、明确的任务指示    
- B. 补充足够的背景信息     
- C. 如有特定用户群体或特定任务要求，设定角色或受众     
- D. 通过思维链提示帮助推理过程更为严谨可靠   
- E. 当描述太抽象或无法准确描述时，可以通过增加示例来明确这些信息 

**【点击查看答案】**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **参考答案：ABCE**  
📝 **解析**：
- 使用推理模型时应尽量避免使用思维链，这可能反而降低推理效果

</div>
</details>

## ✉️ 评价反馈

感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。